# 0 环境配置

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy
from keras.utils import multi_gpu_model
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [2]:
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (378, 504)
modelname = basemodel_name + '_20180124'

# 2 数据生成

In [3]:
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

In [4]:
train_batch_size = 48
valid_batch_size = 2

In [5]:
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    channel_shift_range=50.,
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                shuffle=True, batch_size=train_batch_size)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          shuffle=True, batch_size=valid_batch_size)

Found 1691 images.
Found 188 images.


In [6]:
train_samples_epoch = train_generator.samples
print("samples_train_epoch = {}".format(train_samples_epoch))

steps_train = train_samples_epoch//train_batch_size + 1
print("steps_train = {}".format(steps_train))

valid_samples_epoch = valid_generator.samples
print("samples_valid_epoch = {}".format(valid_samples_epoch))

steps_valid = valid_samples_epoch//valid_batch_size + 1
print("steps_valid = {}".format(steps_valid))

samples_train_epoch = 1691
steps_train = 36
samples_valid_epoch = 188
steps_valid = 95


# 3 搭建模型

In [7]:
model = coremodel(basemodel_name, model_image_size)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 378, 504, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 188, 251, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 188, 251, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 188, 251, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
model = multi_gpu_model(model, gpus=2)

# 4 训练模型

In [10]:
# modify myloss, use pure version
model.compile(optimizer=Adam(lr=1e-3), 
              loss=myloss(diff_weight=0, count_weight=1),
              metrics=[count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
36/36 [==============================] - 49s 1s/step - loss: 4.8557 - _count_accuracy: 0.0156 - val_loss: 33554454.3398 - val__count_accuracy: 0.0000e+00
Epoch 2/50
36/36 [==============================] - 36s 988ms/step - loss: 2.0836 - _count_accuracy: 0.0307 - val_loss: 2.4088 - val__count_accuracy: 0.0158
Epoch 3/50
36/36 [==============================] - 36s 1s/step - loss: 1.4569 - _count_accuracy: 0.0689 - val_loss: 1.5747 - val__count_accuracy: 0.0737
Epoch 4/50
36/36 [==============================] - 36s 998ms/step - loss: 1.1562 - _count_accuracy: 0.1262 - val_loss: 1.7035 - val__count_accuracy: 0.0579
Epoch 5/50
36/36 [==============================] - 37s 1s/step - loss: 1.0232 - _count_accuracy: 0.1669 - val_loss: 1.9668 - val__count_accuracy: 0.0421
Epoch 6/50
36/36 [==============================] - 36s 994ms/step - loss: 0.8733 - _count_accuracy: 0.2286 - val_loss: 0.8272 - val__count_accuracy: 0.2316
Epoch 7/50
36/36 [==============================] - 36s 

In [23]:
model.save_weights("savemodel/"+modelname+"-1.h5")
print("model saved!")

model saved!


## （2） epoch 2，学习率1e-5，使用Adam

In [1]:
# modify myloss, use pure version
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0, count_weight=1),
              metrics=[diff_accuracy(), count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

NameError: name 'model' is not defined

## （3） epoch 3，学习率1e-6，使用RMSprop

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid_sample)

In [ ]:
model.save("models/"+modelname+"-3.h5")
print("model saved!")

## （4） epoch 4，学习率1e-6，使用RMSprop

In [ ]:
model.compile(optimizer=RMSprop(lr=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=steps_train_sample, epochs=1,
                   validation_data=valid_generator, validation_steps=steps_valid_sample)

In [ ]:
model.save("models/"+modelname+"-4.h5")
print("model saved!")

# 5 测试模型

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy

import tensorflow as tf
from keras.models import load_model
print("import done")

# test
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (480, 640)
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError


modelname = basemodel_name + '_20180122'

model = coremodel(basemodel_name, model_image_size)
model.load_weights("savemodel/"+modelname+".h5")

Using TensorFlow backend.


import done
steps_train = 21
steps_valid = 11
Found 20 images.
Found 20 images.
Found 20 images.


In [2]:
gen = MyImageDataGenerator(preprocessing_function=preprocess_input,
                            channel_shift_range=50.,)
test_batch_size = 1
test_generator = gen.myflow_from_directory(os.path.join(basedir,"test"),  model_image_size,
                                         shuffle=False, batch_size=test_batch_size,
                                          save_to_dir='transimg/',
                                          save_prefix='t-',
                                          save_format='jpg')
steps_test = test_generator.samples//test_batch_size + 1
    

NameError: name 'MyImageDataGenerator' is not defined

In [26]:
test_generator.filenames

['zhuti2_1016_03-21-20.jpg',
 'zhuti2_1030_04-40-35.jpg',
 'zhuti2_153_07-51-34.jpg',
 'zhuti2_19_01-53-45.jpg',
 'zhuti2_236_02-17-19.jpg',
 'zhuti2_280_06-05-58.jpg',
 'zhuti2_280_06-09-13.jpg',
 'zhuti2_285_04-01-34.jpg',
 'zhuti2_344_02-37-50.jpg',
 'zhuti2_348_03-39-50.jpg',
 'zhuti2_369_06-15-44.jpg',
 'zhuti2_418_07-36-25.jpg',
 'zhuti2_512_15-11-17.jpg',
 'zhuti2_589_08-31-19.jpg',
 'zhuti2_58_15-27-01.jpg',
 'zhuti2_633_02-05-45.jpg',
 'zhuti2_645_16-52-22.jpg',
 'zhuti2_690_04-55-29.jpg',
 'zhuti2_752_16-35-38.jpg',
 'zhuti2_771_03-10-41.jpg',
 'zhuti2_771_03-22-21.jpg',
 'zhuti2_808_15-41-45-YB.jpg',
 'zhuti2_832_17-25-00.jpg',
 'zhuti2_850_07-03-45.jpg',
 'zhuti2_866_04-04-01.jpg',
 'zhuti2_86_15-57-28.jpg',
 'zhuti2_86_16-00-20.jpg',
 'zhuti2_929_05-39-25.jpg',
 'zhuti2_932_02-31-56.jpg',
 'zhuti2_973_06-27-48.jpg',
 'zhuti5_1017_09-24-06.jpg',
 'zhuti5_1068_05-42-08.jpg',
 'zhuti5_187_03-14-21.jpg',
 'zhuti5_195_03-08-12.jpg',
 'zhuti5_249_05-55-08-YB.jpg',
 'zhuti5_263_0

In [20]:
from postprocess import postprocess

In [21]:
pred = model.predict_generator(test_generator, steps=steps_test)

In [25]:
model.evaluate_generator(test_generator, steps=steps_test)

[0.04730787823912882, 0.9375, 0.8541666666666666]

In [10]:
output = open("pred.npy", 'wb')

In [11]:
np.save(output, pred)

In [12]:
output2 = open("pred.npy", 'rb')

In [13]:
pred = np.load(output2)

In [14]:
for i in range(test_generator.samples):
    x, y = test_generator.next()

In [9]:
for i in range(test_generator.samples):
    x, y = test_generator.next()
    pred = model.predict(x)
    out = postprocess(pred[0])
    print(test_generator.filenames[i])
    y1 = y[0][:7].astype(int)
    print(y1)
    p1 = out[0]
    print(p1)
    y2 = y[0][7:].astype(int)
    print(y2)
    p2 = out[1]
    print(p2)
    flag = ( (y1 == p1).all() and (y2 == p2).all())
    print(flag)

InvalidArgumentError: Current implementation only supports equal length strides in the row and column dimensions.
	 [[Node: separable_conv2d_1/separable_conv2d/depthwise = DepthwiseConv2dNative[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 10, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](mixed10/concat, separable_conv2d_1/depthwise_kernel/read)]]

Caused by op 'separable_conv2d_1/separable_conv2d/depthwise', defined at:
  File "/home/ubuntu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/traitlets/config/application.py", line 653

In [24]:
for x in pred:
    
    out,out1,out2 = postprocess(x)
    
    print(out)
    print(out1)
    print(out2)

[0 0 0 0 0 0 0]
[4 4 3 4 3 3 3]
False
[0 0 0 0 0 0 0]
[0 0 0 0 1 0 0]
False
[0 0 0 0 0 0 0]
[5 5 6 6 8 7 5]
False
[0 0 0 0 0 0 0]
[2 2 2 3 2 2 3]
False
[0 0 0 0 0 0 0]
[3 3 7 4 5 4 5]
False
[0 0 0 0 0 0 0]
[2 2 2 2 2 2 2]
False
[0 0 0 0 0 0 0]
[6 6 6 6 3 7 7]
False
[0 0 0 0 0 0 0]
[3 3 5 4 6 5 5]
False
[0 0 0 0 0 0 0]
[5 4 7 7 4 4 5]
False
[0 0 0 0 0 0 0]
[2 2 4 3 3 3 3]
False
[0 0 0 0 0 0 0]
[2 2 2 2 5 2 1]
False
[0 0 0 0 0 0 0]
[2 2 4 3 2 4 3]
False
[0 0 0 0 0 0 0]
[2 1 0 1 1 1 1]
False
[0 0 0 0 0 0 0]
[0 1 2 0 1 1 1]
False
[0 0 0 0 0 0 0]
[4 3 4 5 6 6 6]
False
[0 0 0 0 0 0 0]
[1 1 1 2 1 2 2]
False
[0 0 0 0 0 0 0]
[4 4 4 6 6 6 6]
False
[0 0 0 0 0 0 0]
[1 1 0 0 0 0 0]
False
[0 0 0 0 0 0 0]
[2 1 2 2 2 1 1]
False
[0 0 0 0 0 0 0]
[0 0 0 1 1 1 1]
False
[0 0 0 0 0 0 0]
[1 1 1 1 2 2 2]
False
[0 0 0 0 0 0 0]
[2 2 2 3 3 4 4]
False
[0 0 0 0 0 0 0]
[4 7 8 6 2 2 3]
False
[0 0 0 0 0 0 0]
[1 1 3 2 1 1 1]
False
[0 0 0 0 0 0 0]
[2 2 2 2 2 2 2]
False
[0 0 0 0 0 0 0]
[2 2 2 1 1 1 3]
False
[0 0 0 0 0 0

In [16]:
out1[0]

1.0